<a href="https://colab.research.google.com/github/pokem1402/CSED703/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Loading Dataset

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os

os.chdir("drive/My Drive/dataset")

In [11]:
!ls

vgames2.csv


In [12]:
import pandas as pd

df_original = pd.read_csv("vgames2.csv")

# 2. EDA

In [13]:
df_original.head()

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


## 1) 데이터에 있는 이상치 발견 및 수정

In [13]:
"""
DataFrame.iterrows()
 : row 값에 대해 loop를 수행하게 해주는 함수.
"""

'\nDataFrame.iterrows()\n\n : row 값에 대해 loop를 수행하게 해주는 함수.\n'

In [14]:
# (1) 불필요한 index 제거
df = df_original.drop(axis=1, columns="Unnamed: 0")

In [15]:
# (2) Year를 Int 형식으로 바꿀 수 없는 이유를 찾고, 수정
import numpy as np
print("The number of na : ", df.Year.isna().sum())
# df.Year.dropna().astype("int") # nan 만 존재.
nan_list = [[idx, df.loc[idx]["Name"], df.loc[idx]["Platform"]] for idx, _ in enumerate(df.Year.isna()) if _] # nan game list

The number of na :  271


### (2) nan 값이 들어있는 데이터의 처리 방식

1.   삭제 (dropna)
2.   관련 데이터를 찾아서 해당 내용을 수정하기 (modify)
3.   무시 (ignore)

In [16]:
# (2) 1. 삭제를 할 경우 데이터가 얼마나 줄어들까.
print(df.isna().any(axis=1).sum())

357


In [17]:
# (2) 2. 어느 attribute에 nan 값이 존재하는가
print(df.isna().any(axis=0))

Name           False
Platform       False
Year            True
Genre           True
Publisher       True
NA_Sales       False
EU_Sales       False
JP_Sales       False
Other_Sales    False
dtype: bool


#### Database에서 이름을 입력하면 Year와 Publisher를 얻는 함수

In [18]:
import re
import lxml.html as lh
import requests
import traceback
import sys
import time

def get_data_from_web(game_name, Year = True, Publisher = True, Genre = True, Debug=False, sleep_time = 10, wait_count = 5):
  print(game_name, end = ' ')

  return_value = {"Name":game_name, "Complete" : False, "error": False}
  try:
    if(Year or Publisher or Genre):

      game_name_ = game_name.replace(" ", "+")

      game_base_url = "https://www.vgchartz.com/game/"
      base_url = "https://www.vgchartz.com/games/games.php?name="
      search_attr ="&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=50&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showpalsales=0&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showlastupdate=0&showothersales=0&showshipped=0"
      resp = requests.get(base_url+game_name+search_attr)
      
      count = 0
      while(not resp.ok):
        print("not ok")
        time.sleep(sleep_time)
        resp = requests.get(base_url+game_name+search_attr)
        count+=1
        if(count > wait_count):
          return return_value

      doc = lh.fromstring(resp.content)

      def get_str(element):
        s = str(element.text_content())
        s = re.sub(r"(\s\s+)", "", s)
        return s 

      def get_year(year):
        return int(('19' if int(year) > 70 else '20') + year)

      names = doc.xpath('//tr/td[3]')
      idx = -1;
      for i,name in enumerate(names[4:]):
        name_ = get_str(name)
        if(name_ == game_name):
          idx = i
          break

      flag_year = False
      if Year:
        years = doc.xpath('//tr/td[6]')
        year = get_str(years[idx+3])
        year_ = year[-2:]
        if(year_ != "/A"):
          year_ = get_year(year_)
          return_value["Year"] = year_ 
          print(year_, end = ' ')
        else:
          flag_year=True

      if Publisher:
        publishers = doc.xpath('//tr/td[5]')
        publisher = get_str(publishers[idx+3])
        return_value["Publisher"] = publisher
        print(publisher, end = ' ')

      if Genre or flag_year:
        hrefs = doc.xpath('//tr/td[3]/a/@href')
        game_name_ = game_name.lower()
        game_name_ = re.sub("[!:\\(\\)']", "", game_name_)
        game_name_ = re.sub("-", " ", game_name_)
        for href in hrefs:
          href_part = href.split("/") 
          # remove special character
          ss = href_part[5].replace("-", " ")

          if(ss == game_name_):
            print(href, end=" ")
            id = href_part[4]
            break
        
        game_info_base_url = "https://www.vgchartz.com/game/"
        resp_ = requests.get(game_info_base_url+id+"/")
        

        count = 0
        while(not resp_.ok):
          print("not ok")
          time.sleep(sleep_time)
          resp = requests.get(base_url+game_name+search_attr)
          count+=1
          if(count > wait_count):
            return return_value
        
        doc_ = lh.fromstring(resp_.content)
        txt = doc_.text_content()
        start = txt.find("Genre")+len("Genre")
        if Genre:
          end = txt.find("Other Versions")
          end = end if end != -1 else txt.find("Release Dates", start)
          genre = txt[start:end]
          return_value["Genre"] = genre
          print(genre, end = ' ')
        if flag_year:
          start = txt.find("Release Dates",start)+len("Release Dates")
          end = txt.find("Community Stats")
          years_ = re.findall("\d\d\/\d\d\/\d\d", txt[start:end])
          year = get_year(str(min([int(y_[-2:]) for y_ in years_])))
          return_value["Year"] = year_
          print(year_, end=' ')
      return_value["Complete"] = True
      return return_value
  except BaseException as e:
    # Get current system exception
    ex_type, ex_value, ex_traceback = sys.exc_info()

    # Extract unformatter stack traces as tuples
    trace_back = traceback.extract_tb(ex_traceback)

    # Format stacktrace
    stack_trace = list()

    for trace in trace_back:
        stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))

    if(Debug):
      print("Exception type : %s " % ex_type.__name__)
      print("Exception message : %s" %ex_value)
      print("Stack trace : %s" %stack_trace)

    else:
      return_value["error"]= True
      return_value["error_msg"] = []
      return_value["error_msg"].append("Exception type : %s " % ex_type.__name__)
      return_value["error_msg"].append("Exception message : %s" %ex_value)
      return_value["error_msg"].append("Stack trace : %s" %stack_trace)
      return return_value
# How to use
# get_data_from_web("Rock Band",Year=True, Publisher=True, Genre = True)
get_data_from_web("My Healthy Cooking Coach", Year=True, Publisher=True, Genre = True, Debug=True);

My Healthy Cooking Coach 2009 Ubisoft https://www.vgchartz.com/game/33485/my-healthy-cooking-coach/?region=All Simulation 

In [19]:
nan_idx = df[df.isna().any(axis=1)].index
nan_idx

Int64Index([   31,    50,   109,   272,   273,   358,   408,   430,   548,
              675,
            ...
            16130, 16195, 16240, 16275, 16278, 16425, 16431, 16432, 16555,
            16587],
           dtype='int64', length=357)

In [33]:
print(np.nan)
np.isnan(df.loc[nan_idx[0]].Year)

nan


True

In [38]:
print(True and True)
print(True * False)
print(False * False)
print(False * True)

True
0
0
0


In [41]:
a = {}

True and "k" not in a.keys()

True

In [ ]:

def find_missing_value(searched_values, missed_idx):

  n = len(searched_values)

  for i, idx in enumerate(missed_idx):
    print(i, idx, end=" ")

    Year = np.isnan(df.loc[idx].Year)
    Publisher = np.isnan(df.loc[idx].Publisher)
    Genre = np.isnan(df.loc[idx].Genre)

    if (i+1 > n): # not visited : append
      value = get_data_from_web(df.loc[idx].Name,
                                Year = Year,
                                Publisher = Publisher,
                                Genre = Genre,
                                Debug=True)

      searched_values.append([idx, value])
      n += 1
    else: # already visited

      # Year : nan and if year value is found already -> no need to find
      # Year : nan and if year value is not found -> we need to find
      Year = Year and ("Year" not in searched_values[i].keys())


    print("")



31
Rock Band
https://www.vgchartz.com/game/228764/rock-band/?region=All
50
Ape Escape: On the Loose
https://www.vgchartz.com/game/6802/ape-escape-on-the-loose/?region=All
109
Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales)
https://www.vgchartz.com/game/35543/yu-gi-oh-5ds-wheelie-breakers-jp-sales/?region=All
272
UK Truck Simulator
https://www.vgchartz.com/game/62448/uk-truck-simulator/?region=All
273
Jewel Link Chronicles: Mountains of Madness
https://www.vgchartz.com/game/70159/jewel-link-chronicles-mountains-of-madness/?region=All
358
Majesty 2: The Fantasy Kingdom Sim
https://www.vgchartz.com/game/33992/majesty-2-the-fantasy-kingdom-sim/?region=All
408
NBA Ballers
https://www.vgchartz.com/game/5440/nba-ballers/?region=All
430
Famista 64
https://www.vgchartz.com/game/3272/famista-64/?region=All
548
Transworld Surf
https://www.vgchartz.com/game/5941/transworld-surf/?region=All
675
Battle vs. Chess
Exception type : ValueError 
Exception message : invalid literal for int() with base 10: ''
S

In [ ]:
missed_value

In [ ]:
df.loc[430]